#### This notebook is created to perform sequence alignment using Smith-Waterman algorithm

The Smith-Waterman algorithm compares the gene fragments of all possible lengths and measure similarity.

#### Import requered modules and libraries

In [1]:
! pip install biopython

In [7]:
import numpy as np
import pylab
import sys
from Bio import SeqIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import Phylo

### Let's compare human and mice IL2 partial gene sequence

In [8]:
Homo_sapiens = "CTATCACCTAAGTGTGGGCTAATGTAACAAAGAGGGATTTCACCTACATCCATTCAGTCAGTCTTTGGGGGTTTAAAGAAATTCCAAAGAGTCATCAGAA"
Mus_musculus = "AACTAGAGACATATAAAATAACACCAACATCCTTAGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAGCATAATAAACATCAAG"

#### Local alignment

In [9]:
Seq1 = Homo_sapiens
Seq2 = Mus_musculus

class Scores(object):
    gap = -10
    match = 6
    mismatch = -1

    def __init__(self, s1, s2):      
        if len(s1) >= len(s2):
            self.t = s1
            self.s = s2
        else:
            self.t = s2
            self.s = s1
        self.a = None
        self.alignments = None   

    def matrix(self):
        m = len(self.s) + 1  
        n = len(self.t) + 1  

        self.a = [
            [0 for j in range(n)] for i in range(m)
        ]

        for i in range(1, m):
            for j in range(1, n):
                value = self.match if self.s[i - 1] == self.t[j - 1] else self.mismatch
                self.a[i][j] = max(
                    self.a[i - 1][j] + self.gap,
                    self.a[i - 1][j - 1] + value,
                    self.a[i][j - 1] + self.gap,
                    0
                )

    def alignment(self, i, j, s, t):

        if self.a[i][j] == 0:
            self.alignments.append(
                [t[::-1], s[::-1]]
            )
            return
        if i > 0 and self.a[i][j] == self.a[i - 1][j] + self.gap:
            self.alignment(
                i=i - 1,
                j=j,
                s=s + self.s[i - 1],
                t=t + '_'
            )
        if i > 0 and j > 0 and self.a[i][j] == self.a[i - 1][j - 1] + (self.match if self.s[i - 1] == self.t[j - 1] else self.mismatch):
            self.alignment(
                i=i - 1,
                j=j - 1,
                s=s + self.s[i - 1],
                t=t + self.t[j - 1]
            )
        if j > 0 and self.a[i][j] == self.a[i][j - 1] + self.gap:
            self.alignment(
                i=i,
                j=j - 1,
                s=s + '_',
                t=t + self.t[j - 1]
            )

    def print_alignments(self):    
        self.alignments = []

        bigger_positions = []
        bigger = -sys.maxsize
        for i in range(len(self.a)):
            for j in range(len(self.a[i])):
                if self.a[i][j] > bigger:
                    bigger = self.a[i][j]
                    bigger_positions = [[i, j]]
                elif self.a[i][j] == bigger:
                    bigger_positions.append([i, j])

        for [i, j] in bigger_positions:
            self.alignment(
                i=i,
                j=j,
                s='',
                t=''
            )
        print('Score: ', bigger, '\n')
        for [t, s] in self.alignments:
            print('Align1: ', t)
            print('Align2: ', s, '\n')

if __name__ == '__main__':
    Scores = Scores(Seq1,Seq2)

    print('\nLocal alignment')
    Scores.matrix()
    Scores.print_alignments()


Local alignment
Score:  218 

Align1:  AACTA_GAGACATATAAAATAACACCAACATCCTT_AGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAG_CATAATAA
Align2:  ACCTAAGTGTGGGCTAATGTAACAAAGAGGGATTTCACCTACATCCATT_CAGTCAGTCTTTGGGGGTTTAAAGAAATTCCAAAGAGTCATCAGAA 

Align1:  AACT_AGAGACATATAAAATAACACCAACATCCTT_AGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAG_CATAATAA
Align2:  ACCTAAGTGTGGGCTAATGTAACAAAGAGGGATTTCACCTACATCCATT_CAGTCAGTCTTTGGGGGTTTAAAGAAATTCCAAAGAGTCATCAGAA 

Align1:  AACTA_GAGACATATAAAATAACACCAACATCCTT_AGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAG_CATAATAA
Align2:  ACCTAAGTGTGGGCTAATGTAACAAAGAGGGATTTCACCTACATCCATTC_AGTCAGTCTTTGGGGGTTTAAAGAAATTCCAAAGAGTCATCAGAA 

Align1:  AACT_AGAGACATATAAAATAACACCAACATCCTT_AGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAG_CATAATAA
Align2:  ACCTAAGTGTGGGCTAATGTAACAAAGAGGGATTTCACCTACATCCATTC_AGTCAGTCTTTGGGGGTTTAAAGAAATTCCAAAGAGTCATCAGAA 

Align1:  AACTA_GAGACATATAAAATAACACCAACATCCTT_AGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAG_CATAATAA
Align2:

#### Global alignment

#### Consider two sequences given below. We want to find out all the possible global alignments with the maximum similarity score.

In [10]:
# No parameters. Identical characters have a score of 1, else 0.
# No gap penalties.
alignments = pairwise2.align.globalxx(Seq1, Seq2)

# format_alignment method to format the alignments in the list
for a in alignments:
    print(format_alignment(*a))

CTATCACCTAAGTGTGG-GCTA-ATGTA-ACAAAG-AGGGATTTCACCTA-CATCCATTCAG-T-CAGT-C--TT--TGGGG-G--TTTAAA--GA-A-AT--T-C-C-----AAAGA-GTCAT------CAG--AA-
  |  | || |     | |  | |  || | |||  |   |   |||| | ||||| || || | ||   |  ||  |   | |  |||  |  |  | ||  | | |     ||| | | |||      ||   || 
--A--A-CT-A-----GAG--ACA--TATA-AAA-TA---A---CACC-AACATCC-TT-AGATACA--ACCCTTCCT---GAGAATTT--ATTG-GACATCATACTCTTTTTAAA-AAG-CATAATAAACA-TCAAG
  Score=63

CTATCACCTAAGTGTGG-GCTA-ATGTA-ACAAAG-AGGGATTTCACCTA-CATCCATTCAG-T-CAGT-C--TT--TGGGG-G--TTTAAA--GA-A-AT--T-C-C-----AAAGA-GTCAT------CAG--AA-
  |  || | |     | |  | |  || | |||  |   |   |||| | ||||| || || | ||   |  ||  |   | |  |||  |  |  | ||  | | |     ||| | | |||      ||   || 
--A--AC-T-A-----GAG--ACA--TATA-AAA-TA---A---CACC-AACATCC-TT-AGATACA--ACCCTTCCT---GAGAATTT--ATTG-GACATCATACTCTTTTTAAA-AAG-CATAATAAACA-TCAAG
  Score=63

CTATCACCTAAGTGTGG-GCTA-ATGTA-ACAAAG-AGGGATTTCACCTA-CATCCATTCAG-T-CAGT-C--TT--TGGGG-G--TTTAAA--GA-A-AT--T-C-C-----AAAGA-GTCAT------CAG--AA-
  |

#### Now we will change the scoring scheme and assign values for matches, gaps and mismatches. Let's try to find all the possible global alignments with the maximum similarity score.
Matching characters = 1 points,deduction of 0.5 point for each mismatching character. 0.25 points deduction when opening a gap, and 0.1 points are deducted when extending it.

In [11]:
# A match score = score of identical chars, else mismatch score.
# Same open and extend gap penalties for both Se11 and Seq2 sequences.
alignments = pairwise2.align.globalms(Seq1, Seq2, 1, -0.5, -0.25, -0.1)

# Use format_alignment method to format the alignments
for a in alignments:
    print(format_alignment(*a))

CTATCACCTAAGTGTGG-G-C-T----AATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC-----ATT-------CAGTCAGT-CT-TTGGGGGTTT--AAAG-A-AATTCCAAAGAGTCATCAGAA-
  |  | |||      | | | |    ||  |||||  ||      ||  ||     ||  |||   |||     |||       || ||| | || ||     |||  |||| | |||   |||    ||||  || 
--A--A-CTA------GAGACATATAAAA--TAACACCAA-----CATCCTTAGATACAACCCT---TCCTGAGAATTTATTGGACA-TCA-TACTCTT-----TTTAAAAAGCATAAT---AAA----CATC--AAG
  Score=51

CTATCACCTAAGTGTGG-G-C-T----AATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC-----ATT-------CAGTCAGT-CT-TTGGGGGTTT--AAAG-A-AATTCCAAAGAGTCATCAGAA-
  |  || ||      | | | |    ||  |||||  ||      ||  ||     ||  |||   |||     |||       || ||| | || ||     |||  |||| | |||   |||    ||||  || 
--A--AC-TA------GAGACATATAAAA--TAACACCAA-----CATCCTTAGATACAACCCT---TCCTGAGAATTTATTGGACA-TCA-TACTCTT-----TTTAAAAAGCATAAT---AAA----CATC--AAG
  Score=51

CTATCACCTA-AGTGTGGG-C-T----AATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC-----ATT-------CAGTCAGT-CT-TTGGGGGTTT--AAAG-A-AATTCCAAAGAGTCATCAGAA-
  |


CTATCACCTA-AGTGTGGG-C-TA----ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC---------ATT---CAGTCAGT-CT-TTGGGGGTTTAA--AG-A-AATTCCAAAGAGTCATCAGAA-
  |  || || ||       | ||    ||  ||||  ||      ||  ||     ||  |||   |||         |||   || ||| | || ||     |||||  || | |||   |||    ||||  || 
--A--AC-TAGAG------ACATATAAAAT--AACACCAA-----CATCCTTAGATACAACCCT---TCCTGAGAATTTATTGGACA-TCA-TACTCTT-----TTTAAAAAGCATAAT---AAA----CATC--AAG
  Score=51

CTATCACCTA-AG---TGTGGGC-TA--ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC---------ATT---CAGTCAGT-CT-TTGGGGGTTTAA--AG-A-AATTCCAAAGAGTCATCAGAA-
  |  | ||| ||   |       ||  ||  ||||  ||      ||  ||     ||  |||   |||         |||   || ||| | || ||     |||||  || | |||   |||    ||||  || 
--A--A-CTAGAGACAT------ATAAAAT--AACACCAA-----CATCCTTAGATACAACCCT---TCCTGAGAATTTATTGGACA-TCA-TACTCTT-----TTTAAAAAGCATAAT---AAA----CATC--AAG
  Score=51

CTATCACCTA-AG---TGTGGGC-TA--ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC---------ATT---CAGTCAGT-CT-TTGGGGGTTTAA--AG-A-AATTCCAAAGAGTCATCAGAA-
  


CTATCACCTA-AGTGTGGG-C---TA--ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC---------ATT---CAGTCAGT-CT-TTGGGGGTTTAAAGAA--AT--TCCAAAGAGTCATCAGAA-
  |  | ||| |      | |   ||  |  |||||  ||      ||  ||     ||  |||   |||         |||   || ||| | || ||     |||||| ||  ||  |  |||    ||||  || 
--A--A-CTAGA------GACATATAAAA--TAACACCAA-----CATCCTTAGATACAACCCT---TCCTGAGAATTTATTGGACA-TCA-TACTCTT-----TTTAAA-AAGCATAAT--AAA----CATC--AAG
  Score=51

CTATCACCTA-AGTGTGGG-C---TA--ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC---------ATT---CAGTCAGT-CT-TTGGGGGTTTAAAGAA--AT--TCCAAAGAGTCATCAGAA-
  |  || || |      | |   ||  |  |||||  ||      ||  ||     ||  |||   |||         |||   || ||| | || ||     |||||| ||  ||  |  |||    ||||  || 
--A--AC-TAGA------GACATATAAAA--TAACACCAA-----CATCCTTAGATACAACCCT---TCCTGAGAATTTATTGGACA-TCA-TACTCTT-----TTTAAA-AAGCATAAT--AAA----CATC--AAG
  Score=51

CTATCACCTA-AGTGTGGG-C---TA--ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC---------ATT---CAGTCAGT-CT-TTGGGGGTTTAAAGAA--AT--TCCAAAGAGTCATCAGAA-
  


CTATCACCTAAGTGTGG-G-C---TA--ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC-----ATT-------CAGTCAGT-CT-TTGGGGGTTTA--AAG-A-AATTCCAAAGAGTCATCAGAA-
  |  || ||      | | |   ||  ||  ||||  ||      ||  ||     ||  |||   |||     |||       || ||| | || ||     ||||  ||| | |||   |||    |||||  | 
--A--AC-TA------GAGACATATAAAAT--AACACCAA-----CATCCTTAGATACAACCCT---TCCTGAGAATTTATTGGACA-TCA-TACTCTT-----TTTAAAAAGCATAAT---AAA----CATCA--AG
  Score=51

CTATCACCTA-AGTGTGGG-C---TA--ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC-----ATT-------CAGTCAGT-CT-TTGGGGGTTTA--AAG-A-AATTCCAAAGAGTCATCAGAA-
  |  | ||| |      | |   ||  ||  ||||  ||      ||  ||     ||  |||   |||     |||       || ||| | || ||     ||||  ||| | |||   |||    |||||  | 
--A--A-CTAGA------GACATATAAAAT--AACACCAA-----CATCCTTAGATACAACCCT---TCCTGAGAATTTATTGGACA-TCA-TACTCTT-----TTTAAAAAGCATAAT---AAA----CATCA--AG
  Score=51

CTATCACCTA-AGTGTGGG-C---TA--ATGTAACA--AAGAGGG-AT--TT-----CA--CCTACATCC-----ATT-------CAGTCAGT-CT-TTGGGGGTTTA--AAG-A-AATTCCAAAGAGTCATCAGAA-
  

#### Let's try biopython
That allows to express concepts in fewer lines of code than would be possible in other languages.

In [12]:
alignments = pairwise2.align.localms(Seq1,Seq2,6,-1,-10,0)

for a in alignments: 
    print(format_alignment(*a))

10 AAGTGTGGGCTAATGTAACAAAGAGGGATTTCACCTACATCCATTCAGTCAGTCTT---TGGGGGTTTAAAGAA----------ATTCCAAAGAGTCATCAGAA
   ||.|...|.|..||..||.||          ||||.||||||.|..|..||..|.|   ||.|..||||..|.|          .||..|||.|| |||.|.||
 1 AACTAGAGACATATAAAATAA----------CACCAACATCCTTAGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAG-CATAATAA
  Score=223

10 AAGTGTGGGCTAATGTAACAAAGAGGGATTTCACCTACATCCATTCAGTCAGTCTTT---GGGGGTTTAAAGAA----------ATTCCAAAGAGTCATCAGAA
   ||.|...|.|..||..||.||          ||||.||||||.|..|..||..|.||   |.|..||||..|.|          .||..|||.|| |||.|.||
 1 AACTAGAGACATATAAAATAA----------CACCAACATCCTTAGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAG-CATAATAA
  Score=223

10 AAGTGTGGGCTAATGTAACAAAGAGGGATTTCACCTACATCCATTCAGTCAGTCTT---TGGGGGTTTAAAGAAA----------TTCCAAAGAGTCATCAGAA
   ||.|...|.|..||..||.||          ||||.||||||.|..|..||..|.|   ||.|..||||..|.|.          ||..|||.|| |||.|.||
 1 AACTAGAGACATATAAAATAA----------CACCAACATCCTTAGATACAACCCTTCCTGAGAATTTATTGGACATCATACTCTTTTTAAAAAG-CATAATAA
  